In [64]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import random 
from sklearn.metrics import classification_report
import os
from sklearn.model_selection import train_test_split

import tensorflow as tf

In [65]:
data=pd.read_csv("/content/Reminder_updated_upd.txt",encoding="latin1",delimiter='\t')
data.head()

,Sentence,Tag
0,Fix a reminder for tomorrow's board meeting at...,O O O O B-reminder.date B-reminder.title I-rem...
1,Remind me to fill the google form at 11 PM,O O O O O B-reminder.title I-reminder.title O ...
2,Set a reminder for doubt clearing session at 5...,O O O O B-reminder.title I-reminder.title I-re...
3,Remind me to call Rishi for notes at midnight,O O O B-reminder.title I-reminder.title I-remi...
4,Set a reminder for doctor's appointment at 4 PM,O O O O B-reminder.title I-reminder.title O B-...


In [66]:
data.dropna(inplace=True)

In [67]:
data.shape

(100, 2)

In [68]:
data.head()

,Sentence,Tag
0,Fix a reminder for tomorrow's board meeting at...,O O O O B-reminder.date B-reminder.title I-rem...
1,Remind me to fill the google form at 11 PM,O O O O O B-reminder.title I-reminder.title O ...
2,Set a reminder for doubt clearing session at 5...,O O O O B-reminder.title I-reminder.title I-re...
3,Remind me to call Rishi for notes at midnight,O O O B-reminder.title I-reminder.title I-remi...
4,Set a reminder for doctor's appointment at 4 PM,O O O O B-reminder.title I-reminder.title O B-...


In [69]:
agg_data = data.copy(True)
agg_data['tokenised_sentences']=agg_data['Sentence'].apply(lambda x:x.split())
agg_data['tag_list']=agg_data['Tag'].apply(lambda x:x.split())
agg_data.head()

,Sentence,Tag,tokenised_sentences,tag_list
0,Fix a reminder for tomorrow's board meeting at...,O O O O B-reminder.date B-reminder.title I-rem...,"[Fix, a, reminder, for, tomorrow's, board, mee...","[O, O, O, O, B-reminder.date, B-reminder.title..."
1,Remind me to fill the google form at 11 PM,O O O O O B-reminder.title I-reminder.title O ...,"[Remind, me, to, fill, the, google, form, at, ...","[O, O, O, O, O, B-reminder.title, I-reminder.t..."
2,Set a reminder for doubt clearing session at 5...,O O O O B-reminder.title I-reminder.title I-re...,"[Set, a, reminder, for, doubt, clearing, sessi...","[O, O, O, O, B-reminder.title, I-reminder.titl..."
3,Remind me to call Rishi for notes at midnight,O O O B-reminder.title I-reminder.title I-remi...,"[Remind, me, to, call, Rishi, for, notes, at, ...","[O, O, O, B-reminder.title, I-reminder.title, ..."
4,Set a reminder for doctor's appointment at 4 PM,O O O O B-reminder.title I-reminder.title O B-...,"[Set, a, reminder, for, doctor's, appointment,...","[O, O, O, O, B-reminder.title, I-reminder.titl..."


In [70]:
agg_data['len_sentence']=agg_data['tokenised_sentences'].apply(lambda x:len(x))
agg_data['len_tag']=agg_data['tag_list'].apply(lambda x:len(x))
agg_data['is_equal']=agg_data.apply(lambda row:1 if row['len_sentence']==row['len_tag'] else 0,axis=1)
agg_data['is_equal'].value_counts()

1    100
Name: is_equal, dtype: int64

In [71]:
agg_data.head()

,Sentence,Tag,tokenised_sentences,tag_list,len_sentence,len_tag,is_equal
0,Fix a reminder for tomorrow's board meeting at...,O O O O B-reminder.date B-reminder.title I-rem...,"[Fix, a, reminder, for, tomorrow's, board, mee...","[O, O, O, O, B-reminder.date, B-reminder.title...",10,10,1
1,Remind me to fill the google form at 11 PM,O O O O O B-reminder.title I-reminder.title O ...,"[Remind, me, to, fill, the, google, form, at, ...","[O, O, O, O, O, B-reminder.title, I-reminder.t...",10,10,1
2,Set a reminder for doubt clearing session at 5...,O O O O B-reminder.title I-reminder.title I-re...,"[Set, a, reminder, for, doubt, clearing, sessi...","[O, O, O, O, B-reminder.title, I-reminder.titl...",12,12,1
3,Remind me to call Rishi for notes at midnight,O O O B-reminder.title I-reminder.title I-remi...,"[Remind, me, to, call, Rishi, for, notes, at, ...","[O, O, O, B-reminder.title, I-reminder.title, ...",9,9,1
4,Set a reminder for doctor's appointment at 4 PM,O O O O B-reminder.title I-reminder.title O B-...,"[Set, a, reminder, for, doctor's, appointment,...","[O, O, O, O, B-reminder.title, I-reminder.titl...",9,9,1


In [72]:
agg_data[agg_data['is_equal']==0].head()

,Sentence,Tag,tokenised_sentences,tag_list,len_sentence,len_tag,is_equal


In [73]:
agg_data=agg_data[agg_data['is_equal']!=0]

In [74]:
agg_data.shape

(100, 7)

In [75]:
sentences_list=agg_data['Sentence'].tolist()
tags_list=agg_data['tag_list'].tolist()

print("Number of Sentences in the Data ",len(sentences_list))
print("Are number of Sentences and Tag list equal ",len(sentences_list)==len(tags_list))

Number of Sentences in the Data  100
Are number of Sentences and Tag list equal  True


In [76]:
sentences_list[0]

"Fix a reminder for tomorrow's board meeting at 12 PM"

In [77]:
tags_list[0]

['O',
 'O',
 'O',
 'O',
 'B-reminder.date',
 'B-reminder.title',
 'I-reminder.title',
 'O',
 'B-reminder.time',
 'I-reminder.time']

In [78]:
tokeniser= tf.keras.preprocessing.text.Tokenizer(lower=False,filters='')

tokeniser.fit_on_texts(sentences_list)

In [79]:
print("Vocab size of Tokeniser ",len(tokeniser.word_index)+1) ## Adding one since 0 is reserved for padding

Vocab size of Tokeniser  262


In [80]:
tokeniser.index_word[100]

'text'

In [81]:
encoded_sentence=tokeniser.texts_to_sequences(sentences_list)
print("First Original Sentence ",sentences_list[0])
print("First Encoded Sentence ",encoded_sentence[0])
print("Is Length of Original Sentence Same as Encoded Sentence ",len(sentences_list[0].split())==len(encoded_sentence[0]))
print("Length of First Sentence ",len(encoded_sentence[0]))

First Original Sentence  Fix a reminder for tomorrow's board meeting at 12 PM
First Encoded Sentence  [43, 8, 9, 19, 36, 82, 44, 1, 83, 3]
Is Length of Original Sentence Same as Encoded Sentence  True
Length of First Sentence  10


In [82]:
print(len(list(set(data['Tag'].values))))

65


In [83]:
tagset = set()
for x in data['Tag']:
    for y in x.split():
        tagset.add(y)
print(len(list(tagset)))

7


In [84]:
tags=list(tagset)
#print(tags)
num_tags=len(tags)
print("Number of Tags ",num_tags)

tags_map={tag:i for i,tag in enumerate(tags)}
print("Tags Map ",tags_map)

Number of Tags  7
Tags Map  {'I-reminder.date': 0, 'B-reminder.title': 1, 'I-reminder.title': 2, 'B-reminder.date': 3, 'O': 4, 'I-reminder.time': 5, 'B-reminder.time': 6}


In [85]:
reverse_tag_map={v: k for k, v in tags_map.items()}

In [86]:
print("Reverse Tags Map ", reverse_tag_map)

Reverse Tags Map  {0: 'I-reminder.date', 1: 'B-reminder.title', 2: 'I-reminder.title', 3: 'B-reminder.date', 4: 'O', 5: 'I-reminder.time', 6: 'B-reminder.time'}


In [87]:
encoded_tags=[[tags_map[w] for w in tag] for tag in tags_list]
print("First Sentence ",sentences_list[0])
print('First Sentence Original Tags ',tags_list[0])
print("First Sentence Encoded Tags ",encoded_tags[0])
print("Is length of Original Tags and Encoded Tags same ",len(tags_list[0])==len(encoded_tags[0]))
print("Length of Tags for First Sentence ",len(encoded_tags[0]))

First Sentence  Fix a reminder for tomorrow's board meeting at 12 PM
First Sentence Original Tags  ['O', 'O', 'O', 'O', 'B-reminder.date', 'B-reminder.title', 'I-reminder.title', 'O', 'B-reminder.time', 'I-reminder.time']
First Sentence Encoded Tags  [4, 4, 4, 4, 3, 1, 2, 4, 6, 5]
Is length of Original Tags and Encoded Tags same  True
Length of Tags for First Sentence  10


In [88]:
max_sentence_length=max([len(s.split()) for s in sentences_list])
print(max_sentence_length)

18


In [89]:
max_len=32
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

padded_encoded_sentences=pad_sequences(maxlen=max_len,sequences=encoded_sentence,padding="post",value=0)
padded_encoded_tags=pad_sequences(maxlen=max_len,sequences=encoded_tags,padding="post",value=tags_map['O'])

print("Shape of Encoded Sentence ",padded_encoded_sentences.shape)
print("Shape of Encoded Labels ",padded_encoded_tags.shape)

print("First Encoded Sentence Without Padding ",encoded_sentence[0])
print("First Encoded Sentence with padding ",padded_encoded_sentences[0])
print("First Sentence Encoded Label without Padding ",encoded_tags[0])
print("First Sentence Encoded Label with Padding ",padded_encoded_tags[0])

Shape of Encoded Sentence  (100, 32)
Shape of Encoded Labels  (100, 32)
First Encoded Sentence Without Padding  [43, 8, 9, 19, 36, 82, 44, 1, 83, 3]
First Encoded Sentence with padding  [43  8  9 19 36 82 44  1 83  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0]
First Sentence Encoded Label without Padding  [4, 4, 4, 4, 3, 1, 2, 4, 6, 5]
First Sentence Encoded Label with Padding  [4 4 4 4 3 1 2 4 6 5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]


In [90]:
target= [to_categorical(i,num_classes = num_tags) for i in  padded_encoded_tags]
print("Shape of Labels  after converting to Categorical for first sentence ",target[0].shape)

Shape of Labels  after converting to Categorical for first sentence  (32, 7)


In [91]:
target[0]

array([[0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
 

In [92]:
from sklearn.model_selection import train_test_split
X_train_val,X_test,y_train_val,y_test = train_test_split(padded_encoded_sentences,target,test_size = 0.2,random_state=42)
X_train,X_val,y_train,y_val = train_test_split(X_train_val,y_train_val,test_size = 0.2,random_state=42)
print("Input Train Data Shape ",X_train.shape)
print("Train Labels Length ",len(y_train))
print("Input Test Data Shape ",X_test.shape)
print("Test Labels Length ",len(y_test))

print("Input Validation Data Shape ",X_val.shape)
print("Validation Labels Length ",len(y_val))

Input Train Data Shape  (64, 32)
Train Labels Length  64
Input Test Data Shape  (20, 32)
Test Labels Length  20
Input Validation Data Shape  (16, 32)
Validation Labels Length  16


In [93]:
print("First Sentence in Training Data ",X_train[0])
print("First sentence Label ",y_train[0])
print("Shape of First Sentence -Train",X_train[0].shape)
print("Shape of First Sentence Label  -Train",y_train[0].shape)

First Sentence in Training Data  [ 14  15   6   5   2  64  70   7 233  30   1  27   3   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
First sentence Label  [[0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]]
Shape of First Sentence -Train (32,)
Shape of

In [94]:
from tensorflow.keras import Model,Input
from tensorflow.keras.layers import LSTM,Embedding,Dense
from tensorflow.keras.layers import TimeDistributed, SpatialDropout1D,Bidirectional

In [95]:
embedding_dim=64
vocab_size=len(tokeniser.word_index)+1
lstm_units=64
max_len=32

input_word = Input(shape = (max_len,))
model = Embedding(input_dim = vocab_size+1,output_dim = embedding_dim,input_length = max_len)(input_word)
model = Bidirectional(LSTM(units=embedding_dim, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode = 'concat')(model)
model = LSTM(units=embedding_dim,return_sequences=True)(model)
out = TimeDistributed(Dense(num_tags,activation = 'softmax'))(model)
model = Model(input_word,out)
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 32, 64)            16832     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 32, 128)           66048     
_________________________________________________________________
lstm_5 (LSTM)                (None, 32, 64)            49408     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 32, 7)             455       
Total params: 132,743
Trainable params: 132,743
Non-trainable params: 0
_________________________________________________________________


In [96]:
model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [97]:
history = model.fit(X_train,np.array(y_train),validation_data=(X_val,np.array(y_val)),batch_size = 32,epochs = 100)
model.save('mymodel.h5')

Epoch 1/100
2/2 [==============================] - 6s 978ms/step - loss: 1.9120 - accuracy: 0.6069 - val_loss: 1.8441 - val_accuracy: 0.8047
Epoch 2/100
2/2 [==============================] - 0s 149ms/step - loss: 1.8147 - accuracy: 0.8174 - val_loss: 1.7234 - val_accuracy: 0.8047
Epoch 3/100
2/2 [==============================] - 0s 134ms/step - loss: 1.6751 - accuracy: 0.8174 - val_loss: 1.5334 - val_accuracy: 0.8047
Epoch 4/100
2/2 [==============================] - 0s 137ms/step - loss: 1.4539 - accuracy: 0.8174 - val_loss: 1.2339 - val_accuracy: 0.8047
Epoch 5/100
2/2 [==============================] - 0s 134ms/step - loss: 1.1222 - accuracy: 0.8174 - val_loss: 0.9052 - val_accuracy: 0.8047
Epoch 6/100
2/2 [==============================] - 0s 127ms/step - loss: 0.8377 - accuracy: 0.8174 - val_loss: 0.8522 - val_accuracy: 0.8047
Epoch 7/100
2/2 [==============================] - 0s 135ms/step - loss: 0.8265 - accuracy: 0.8174 - val_loss: 0.9299 - val_accuracy: 0.8047
Epoch 8/100
2

In [98]:
!ls -al /content/

total 1660
drwxr-xr-x 1 root root    4096 Sep  8 16:14 .
drwxr-xr-x 1 root root    4096 Sep  8 15:55 ..
drwxr-xr-x 4 root root    4096 Sep  1 19:26 .config
drwxr-xr-x 2 root root    4096 Sep  8 16:09 .ipynb_checkpoints
-rw-r--r-- 1 root root 1655704 Sep  8 16:14 mymodel.h5
-rw-r--r-- 1 root root   16543 Sep  8 16:09 Reminder_updated_upd.txt
drwxr-xr-x 1 root root    4096 Sep  1 19:26 sample_data


In [99]:
preds=model.predict(X_test) ## Predict using model on Test Data

In [100]:
def evaluatePredictions(test_data,preds,actual_preds):
    print("Shape of Test Data Array",test_data.shape)
    y_actual=np.argmax(np.array(actual_preds),axis=2)
    y_pred=np.argmax(preds,axis=2)
    num_test_data=test_data.shape[0]
    print("Number of Test Data Points ",num_test_data)
    data=pd.DataFrame()
    df_list=[]
    for i in range(num_test_data):
        test_str=list(test_data[i])
        df=pd.DataFrame()
        df['test_input']=test_data[i]
        df['test_tokens']=test_str
        df['tokens']=df['test_tokens'].apply(lambda x:tokeniser.index_word[x] if x!=0 else '<PAD>')
        df['actual_target_index']=list(y_actual[i])
        df['pred_target_index']=list(y_pred[i])
        df['actual_target_tag']=df['actual_target_index'].apply(lambda x:reverse_tag_map[x])
        df['pred_target_tag']=df['pred_target_index'].apply(lambda x:reverse_tag_map[x])
        df['id']=i+1
        df_list.append(df)
    data=pd.concat(df_list)
    pred_data=data[data['tokens']!='<PAD>']
    accuracy=pred_data[pred_data['actual_target_tag']==pred_data['pred_target_tag']].shape[0]/pred_data.shape[0]
    
    return pred_data,accuracy

In [101]:
pred_data,accuracy=evaluatePredictions(X_test,preds,y_test)

Shape of Test Data Array (20, 32)
Number of Test Data Points  20


In [102]:
y_pred=pred_data['pred_target_tag'].tolist()
y_actual=pred_data['actual_target_tag'].tolist()

In [103]:
print(classification_report(y_actual,y_pred))

                  precision    recall  f1-score   support

 B-reminder.date       0.00      0.00      0.00         6
 B-reminder.time       0.61      0.70      0.65        20
B-reminder.title       0.90      0.95      0.93        20
 I-reminder.date       0.00      0.00      0.00         1
 I-reminder.time       0.64      0.47      0.55        19
I-reminder.title       0.96      0.89      0.92        54
               O       0.86      0.96      0.91       104

        accuracy                           0.85       224
       macro avg       0.57      0.57      0.57       224
    weighted avg       0.82      0.85      0.83       224



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [104]:
pred_data[pred_data['actual_target_tag']=="B-reminder.text"]

,test_input,test_tokens,tokens,actual_target_index,pred_target_index,actual_target_tag,pred_target_tag,id


In [105]:
pred_data[pred_data['actual_target_tag']=="B-reminder.date"]

,test_input,test_tokens,tokens,actual_target_index,pred_target_index,actual_target_tag,pred_target_tag,id
9,10,10,tomorrow,3,5,B-reminder.date,I-reminder.time,1
11,10,10,tomorrow,3,4,B-reminder.date,O,6
13,10,10,tomorrow,3,4,B-reminder.date,O,7
4,36,36,tomorrow's,3,1,B-reminder.date,B-reminder.title,10
11,30,30,everyday,3,4,B-reminder.date,O,13
9,10,10,tomorrow,3,5,B-reminder.date,I-reminder.time,15


In [106]:
y_pred[0]

'O'

In [107]:
y_actual[0]

'O'

In [108]:
pred_data['tokens'][1]

1        me
1    should
1    should
1        me
1         a
1    should
1        me
1        me
1        me
1         a
1        me
1        me
1        me
1        me
1        me
1         a
1        me
1        me
1        me
1    should
Name: tokens, dtype: object

In [109]:
X_test[0]

array([ 11,   4,   2, 234,   7, 235,   1,  20,  12,  10,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0], dtype=int32)

In [110]:
test_sentences = tokeniser.sequences_to_texts(X_test)

In [111]:
test_sentences[1]

'I should be reminded to attend calculus class at 3 PM'

In [112]:
y_test[0]

array([[0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
 

In [113]:
preds[0]

array([[1.47948822e-03, 5.77888777e-03, 1.15564140e-03, 3.07676080e-03,
        9.84611332e-01, 1.41142728e-03, 2.48643826e-03],
       [7.19612653e-05, 2.15559569e-03, 9.70008405e-05, 4.39667929e-04,
        9.96773541e-01, 1.27857711e-04, 3.34398152e-04],
       [9.24311898e-05, 3.98874022e-02, 1.29459123e-03, 1.39067043e-03,
        9.55838084e-01, 3.11375392e-04, 1.18538260e-03],
       [2.99909734e-04, 6.24133110e-01, 5.27936816e-02, 6.16618199e-03,
        3.06905568e-01, 1.97311700e-03, 7.72840716e-03],
       [5.93191478e-04, 1.86872482e-01, 7.86877751e-01, 6.07491191e-03,
        1.24565493e-02, 2.26423983e-03, 4.86087007e-03],
       [1.11963961e-03, 2.86841281e-02, 9.47315454e-01, 5.73736150e-03,
        1.13800317e-02, 2.01493781e-03, 3.74846510e-03],
       [3.61723755e-03, 1.51940575e-02, 1.20124027e-01, 3.66572477e-02,
        7.28858471e-01, 2.11418960e-02, 7.44070783e-02],
       [4.39618295e-03, 2.87357182e-03, 1.12149175e-02, 6.14365041e-02,
        1.49081171e-01, 1

In [114]:
pred_data['pred_target_index'][0]

0    4
0    4
0    4
0    4
0    4
0    4
0    4
0    4
0    4
0    4
0    4
0    4
0    4
0    4
0    4
0    4
0    4
0    4
0    4
0    4
Name: pred_target_index, dtype: int64

In [115]:
pred_data['test_input'][0]

0    11
0    14
0    14
0    11
0    13
0    14
0    16
0    11
0    16
0    43
0    16
0    16
0    16
0    16
0    11
0    37
0    11
0    16
0    11
0    14
Name: test_input, dtype: int32

In [116]:
pred_data.shape

(224, 8)

In [117]:
X_test.shape

(20, 32)

In [118]:
fin_data = pred_data.groupby('id')

In [119]:
fin_data.head()

,test_input,test_tokens,tokens,actual_target_index,pred_target_index,actual_target_tag,pred_target_tag,id
0,11,11,Remind,4,4,O,O,1
1,4,4,me,4,4,O,O,1
2,2,2,to,4,4,O,O,1
3,234,234,press,1,1,B-reminder.title,B-reminder.title,1
4,7,7,my,2,2,I-reminder.title,I-reminder.title,1
...,...,...,...,...,...,...,...,...
0,14,14,I,4,4,O,O,20
1,15,15,should,4,4,O,O,20
2,6,6,be,4,4,O,O,20
3,5,5,reminded,4,4,O,O,20


In [120]:
pred_data[pred_data['id']==2]

,test_input,test_tokens,tokens,actual_target_index,pred_target_index,actual_target_tag,pred_target_tag,id
0,14,14,I,4,4,O,O,2
1,15,15,should,4,4,O,O,2
2,6,6,be,4,4,O,O,2
3,5,5,reminded,4,4,O,O,2
4,2,2,to,4,4,O,O,2
5,26,26,attend,1,1,B-reminder.title,B-reminder.title,2
6,176,176,calculus,2,2,I-reminder.title,I-reminder.title,2
7,53,53,class,2,2,I-reminder.title,I-reminder.title,2
8,1,1,at,4,4,O,O,2
9,24,24,3,6,6,B-reminder.time,B-reminder.time,2
